## Describing EMS & Fire Data

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from dotenv import load_dotenv

load_dotenv()

True

In [37]:
ems_example_df = pd.read_csv(os.getenv('EMS_DATA_EXAMPLE_PATH'))
fire_example_df = pd.read_csv(os.getenv('FIRE_DATA_EXAMPLE_PATH'))


# print("EMS DataFrame Info:")
# print(ems_example_df.info())
# print(ems_example_df.describe)

# print("\nFire DataFrame Info:")
# print(fire_example_df.info())
# print(fire_example_df.describe)

#### Average Response Time

In [38]:
fire_example_df["dispatch_time"] = pd.to_datetime(fire_example_df["Apparatus Resource Dispatch Date Time (FD18.3)"], errors="coerce")
fire_example_df["arrive_time"] = pd.to_datetime(fire_example_df["Apparatus Resource Arrival Date Time (FD18.4)"], errors="coerce")

fire_example_df["resp_mins"] = (fire_example_df["arrive_time"] - fire_example_df["dispatch_time"]).dt.total_seconds() / 60

res = (
    fire_example_df.groupby("Basic Incident Type (FD1.21)")["resp_mins"]
    .mean()
    .sort_values(ascending=False)
    .dropna()
)

print("Top 5 Incident Types:")
print(res.head())

# plt.figure(figsize=(10,6))
# res.head(10).plot(kind="barh")
# plt.xlabel("Avg Response Time (mins)")
# plt.ylabel("Incident Type")
# plt.title("Top Incident Types By Avg Response Time")
# plt.gca().invert_yaxis()
# plt.tight_layout()
#plt.show()

Top 5 Incident Types:
Basic Incident Type (FD1.21)
Outside equipment fire                              21.566667
Search for person on land                           15.296726
Brush or brush-and-grass mixture fire (Wildland)    13.733333
Carbon monoxide incident                            13.179365
Sprinkler activation, no fire - unintentional       12.191667
Name: resp_mins, dtype: float64
